In [7]:
import json
import numpy as np
import pandas as pd
from scipy.cluster.hierarchy import linkage, cut_tree, fcluster
import pickle as pkl

In [8]:
with open("data/og/embeddings.json") as f:
    embeddings = json.load(f)

reverse_embeddings = {}
for i in range(len(embeddings.keys())):
    reverse_embeddings[tuple(list(embeddings.values())[i])] = list(embeddings.keys())[i]

with open("data/og/openai_summaries_2", "rb") as fp:
    summaries = pkl.load(fp)

In [9]:
with open('data/og/LinkageLabels2.pkl', 'rb') as f:
    linkage_labels = pkl.load(f)

with open('data/pca5_normalized.json') as f:
    pca5_normalized = json.load(f)

In [10]:
# https://joernhees.de/blog/2015/08/26/scipy-hierarchical-clustering-and-dendrogram-tutorial/
#  Creates value of links, such as color, by blending together leaf values
def create_link_values(linkage_matrix, leaf_values,
                    merge_function = lambda cluster1_val, cluster2_val, weight: cluster1_val * (1 - weight) + cluster2_val * weight,
                    weight_function = lambda cluster1_size, cluster2_size: cluster2_size / (cluster1_size + cluster2_size)):
    # Merge function takes val1, val2, and a weight (0-1, 0 preferences val1, 1 preferences val2) and returns a merged value

    num_leaves = len(leaf_values)
    
    new_values = leaf_values.copy()
    for row in linkage_matrix:
        cluster1_id = int(row[0])
        cluster2_id = int(row[1])

        if cluster1_id < num_leaves: # it is a sample/leaaf
            cluster1_size = 1
        else:
            cluster1_size = linkage_matrix[num_leaves - cluster1_id][-1]
        
        if cluster2_id < num_leaves: # it is a sample/leaf
            cluster2_size = 1
        else:
            cluster2_size = linkage_matrix[num_leaves - cluster2_id][-1]

        # If a cluster is huge and is merged with a leaf, the weight should preference the cluster
        # Can try different weighting here
        weight = weight_function(cluster1_size, cluster2_size)

        # merge the values for each index, 5 indexes
        try:
            merged_cluster_value = [merge_function(new_values[cluster1_id][i], new_values[cluster2_id][i], weight) for i in range(5)]
        except:
            print("INDEX OUT OF RANGE: " + cluster1_id, cluster2_id, weight, cluster1_size, cluster2_size)

        new_values.append(merged_cluster_value)

    return new_values

In [11]:
Z = linkage(list(embeddings.values()))
leaf_values = [list(map(float,i)) for i in list(pca5_normalized.values())]

all_values = create_link_values(Z, leaf_values)
new_cluster_values = all_values[-len(Z):]